In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import re
import sys  
reload(sys)  
sys.setdefaultencoding('utf8')

## 用户订单数据

In [8]:
#原始订单数据
data = pd.read_csv('./data/usr_orders_17w.csv',sep=',') #1.1G
data.drop_duplicates(subset=['uid','ordNum','shopId','ordTime','orderStatus'],inplace=True) #
data.head(),data.shape

(                                uid  huabeiAmount  huabeiTotal  actualFee  \
 0  9db93f9be3cd25216b7f6c6fce89bf9c     1115441.0    1300000.0       88.0   
 1  9db93f9be3cd25216b7f6c6fce89bf9c     1115441.0    1300000.0       58.0   
 2  9db93f9be3cd25216b7f6c6fce89bf9c     1115441.0    1300000.0       98.0   
 3  9db93f9be3cd25216b7f6c6fce89bf9c     1115441.0    1300000.0        8.8   
 4  9db93f9be3cd25216b7f6c6fce89bf9c     1115441.0    1300000.0      108.0   
 
               ordNum postFee   shopNick      shopId              ordTime  \
 0  22966768811974484    0.00    nice1st   673601501  2017-05-25 21:59:16   
 1  22639190096974484    0.00  广源互动母婴专营店  1705258730  2017-05-24 16:10:01   
 2  21128088742974484    0.00     我爱悠享每刻  1739489952  2017-05-19 14:29:21   
 3  22956188751974484    0.00   guohaotj   281147411  2017-05-25 21:21:43   
 4  21137270023974484    0.00     我爱悠享每刻  1739489952  2017-05-19 14:26:21   
 
   orderStatus orderTimeout  quantity    shopName  
 0       卖家已发货

In [9]:
#店铺id缺失值填充
import re
def re_trans(s):
    if isinstance(s,str):
        return re.findall('\d+',s)[0]

def ids_trans(ids):
    t = pd.Series(ids).apply(re_trans).dropna()
    if len(t)>0:
        return t.values[0]
    else:
        return np.nan
shop_ids = data[['shopNick','shopId']].groupby(by=['shopNick'])['shopId'].apply(ids_trans)
shop_ids = shop_ids.reset_index()

In [10]:
#交易关闭数据
data_close = data[(data.orderStatus.str.contains('关闭')) | (data.orderStatus=='等待买家付款')]
#交易成功数据
data.drop(data_close.index,inplace=True)
data['huabeiAmount'] = data['huabeiAmount'].apply(lambda x:x/100)
data['huabeiTotal'] = data['huabeiTotal'].apply(lambda x:x/100)
data.shape

(5383209, 13)

### 1.用户购物行为特征统计

In [11]:
#用户购买金额以及订单商品数统计,商品均价
result1 = data.groupby(by=['uid'])['actualFee','quantity'].agg(['sum','mean','median','std','min','max'])
cols = [i[0]+'_'+i[1] for i in result1.columns]
result1.columns = cols
del cols
result1['good_avg_fee'] = result1['actualFee_sum']/result1['quantity_sum']
result1.shape

(170054, 13)

In [12]:
#订单数,单笔订单购买店铺最多的店铺数,单笔均价,
tmp1 = data.groupby(by=['uid'])['ordTime'] .describe()
result1['order_count'] = tmp1['unique']
result1['order_shop_most'] = tmp1['freq']
result1['order_avg_fee'] = result1['actualFee_sum'] / result1['order_count']
result1.shape

(170054, 16)

In [13]:
#用户购买店铺数,偏好店铺购买次数,偏好店铺id,偏好店铺购买次数占比
data['shopId'] = data['shopId'].astype(str)
tmp1 = data.groupby(by=['uid'])['shopId'].describe()
result1['shop_count'] = tmp1['unique']
result1['shop_freq_num'] = tmp1['freq']
result1['shop_freq_id'] = tmp1['top']
result1['shop_freq_ratio'] = 1.0*result1['shop_freq_num']/result1['order_count']
del tmp1
result1.shape

(170054, 20)

In [24]:
#时间间隔计算函数(单位:天)
def time_gap(times):
    try:
        times = pd.to_datetime(pd.Series(times).drop_duplicates())
        if len(times)==2:
            mean = np.nan
            std = np.nan
        else:
            times = times.sort_values()
            gaps = (times -times.shift()).dropna()
            gaps = gaps.map(lambda x: x/np.timedelta64(1,'D'))
            mean = gaps.mean()
            std = gaps.std()    
        return mean,std
    except:
        print times        

#订单创建时间格式化
def trans_dt(time):
    try:
        j = time.split('(')[0]
        j= pd.to_datetime(j)
        return j
    except:
        print time
data['ordTime'] = data['ordTime'].apply(trans_dt)

In [15]:
#用户购买时间间隔方差均值
data['ordTime'] = data['ordTime'].astype(str)
tmp1 = data.groupby(by=['uid'])['ordTime'].apply(time_gap)

In [10]:
result1['order_time_gapMean'] = tmp1.apply(lambda x:x[0])
result1['order_time_gapStd'] = tmp1.apply(lambda x:x[1])
#缺失值填充
result1['order_time_gapMean'].fillna(result1['order_time_gapMean'].mean(),inplace=True)
result1['order_time_gapStd'].fillna(result1['order_time_gapStd'].mean(),inplace=True)
result1.shape

(170054, 18)

In [16]:
#在线时间(天,年)
def time_start(times):
    times =pd.Series(times)
    times = times.sort_values()
    days = (dt.datetime.today()-pd.to_datetime(times.iat[0])).days
    years = dt.datetime.today().year - pd.to_datetime(times.iat[0]).year
    return days,years+1
data['ordTime'] = data['ordTime'].astype(str)
tmp = data.groupby(by=['uid'])['ordTime'].apply(time_start)
result1['online_days'] = tmp.apply(lambda x:x[0])
result1['online_years'] = tmp.apply(lambda x:x[1])
result1['year_avg_fee'] = result1['actualFee_sum'] / result1['online_years']
result1['year_avg_order'] = 1.0*result1['order_count'] / result1['online_years']
result1.shape

(170054, 24)

In [17]:
#用户交易关闭数据统计,关闭交易占比
tmp = data_close.groupby(by=['uid'])['ordTime'].describe()
result1['order_close_count'] = tmp['unique']
result1['order_close_count'].fillna(0,inplace=True)
result1['order_close_ratio'] = 1.0*result1['order_close_count']/result1['order_count']
result1.shape

(170054, 26)

In [18]:
#免邮订单数,免邮订单率
result1['free_post_count'] = data[(data.postFee==0) | (data.postFee=='0.00') | (data.postFee==np.nan)].groupby(by=['uid'])['actualFee'].count()
result1['free_post_count'].fillna(0,inplace=True)
result1['free_post_ratio'] = 1.0 * result1['free_post_count'] /data.groupby(by=['uid'])['actualFee'].count()
result1.head()

,actualFee_sum,actualFee_mean,actualFee_median,actualFee_std,actualFee_min,actualFee_max,quantity_sum,quantity_mean,quantity_median,quantity_std,...,shop_freq_id,shop_freq_ratio,online_days,online_years,year_avg_fee,year_avg_order,order_close_count,order_close_ratio,free_post_count,free_post_ratio
uid,,,,,,,,,,,,,,,,,,,,,
00006f133dc9233b5476ed47b00aa838,2774.06,71.129744,39.9,76.600683,7.96,399.0,88.0,2.256410,2.0,1.816850,...,2832360567,0.307692,434,2,1387.030000,19.5,9,0.230769,27.0,0.692308
0001101072642d28cbec77255a351fa1,317.60,52.933333,57.4,33.439597,9.90,89.0,12.0,2.000000,2.0,0.632456,...,772114767,0.333333,70,1,317.600000,6,1,0.166667,5.0,0.833333
000120625f722f19873e3e08ef3ee0d7,330.80,55.133333,47.1,45.429095,9.80,133.0,13.0,2.166667,2.0,0.408248,...,2643498115,0.333333,120,1,330.800000,3,0,0,5.0,0.833333
000166ec0b788bb54690c172e5946574,1383.90,44.641935,34.0,35.564498,9.90,158.0,93.0,3.000000,2.0,3.405877,...,2568194073,0.0322581,846,3,461.300000,10.3333,1,0.0322581,21.0,0.677419
0001c5790371fcaec5b9aa8327a905be,5050.85,67.344667,45.0,100.508308,8.50,798.0,114.0,1.520000,1.0,0.577974,...,2170206006,0.0266667,800,3,1683.616667,25,14,0.186667,71.0,0.946667


### 花呗数据


In [19]:
#花呗消费额度,花呗消费率
data['huabei_cost'] = data['huabeiTotal'] - data['huabeiAmount']
data['huabei_cost_ratio'] = data['huabei_cost'] / (data['huabeiAmount']+0.01)
result1['huabei_cost'] = data.groupby(by=['uid'])['huabei_cost'].apply(lambda a:list(a)[0])
result1['huabei_cost_ratio'] = data.groupby(by=['uid'])['huabei_cost_ratio'].apply(lambda a:list(a)[0])
result1['huabeiTotal']  = data.groupby(by=['uid'])['huabeiTotal'].apply(lambda a:list(a)[0])
result1.shape,result1.head()

((170054, 31),
                                   actualFee_sum  actualFee_mean  \
 uid                                                               
 00006f133dc9233b5476ed47b00aa838        2774.06       71.129744   
 0001101072642d28cbec77255a351fa1         317.60       52.933333   
 000120625f722f19873e3e08ef3ee0d7         330.80       55.133333   
 000166ec0b788bb54690c172e5946574        1383.90       44.641935   
 0001c5790371fcaec5b9aa8327a905be        5050.85       67.344667   
 
                                   actualFee_median  actualFee_std  \
 uid                                                                 
 00006f133dc9233b5476ed47b00aa838              39.9      76.600683   
 0001101072642d28cbec77255a351fa1              57.4      33.439597   
 000120625f722f19873e3e08ef3ee0d7              47.1      45.429095   
 000166ec0b788bb54690c172e5946574              34.0      35.564498   
 0001c5790371fcaec5b9aa8327a905be              45.0     100.508308   
 
               

In [20]:
#save
result1.fillna(0,inplace=True)
result1.to_csv('./data/feature_usr_base_17w.csv')
result1.head()

,actualFee_sum,actualFee_mean,actualFee_median,actualFee_std,actualFee_min,actualFee_max,quantity_sum,quantity_mean,quantity_median,quantity_std,...,online_years,year_avg_fee,year_avg_order,order_close_count,order_close_ratio,free_post_count,free_post_ratio,huabei_cost,huabei_cost_ratio,huabeiTotal
uid,,,,,,,,,,,,,,,,,,,,,
00006f133dc9233b5476ed47b00aa838,2774.06,71.129744,39.9,76.600683,7.96,399.0,88.0,2.256410,2.0,1.816850,...,2,1387.030000,19.500000,9,0.230769,27.0,0.692308,0.00,0.000000,0.0
0001101072642d28cbec77255a351fa1,317.60,52.933333,57.4,33.439597,9.90,89.0,12.0,2.000000,2.0,0.632456,...,1,317.600000,6.000000,1,0.166667,5.0,0.833333,215.57,0.757875,500.0
000120625f722f19873e3e08ef3ee0d7,330.80,55.133333,47.1,45.429095,9.80,133.0,13.0,2.166667,2.0,0.408248,...,1,330.800000,3.000000,0,0.000000,5.0,0.833333,17.21,0.524695,50.0
000166ec0b788bb54690c172e5946574,1383.90,44.641935,34.0,35.564498,9.90,158.0,93.0,3.000000,2.0,3.405877,...,3,461.300000,10.333333,1,0.032258,21.0,0.677419,0.00,0.000000,0.0
0001c5790371fcaec5b9aa8327a905be,5050.85,67.344667,45.0,100.508308,8.50,798.0,114.0,1.520000,1.0,0.577974,...,3,1683.616667,25.000000,14,0.186667,71.0,0.946667,0.00,0.000000,4000.0


In [25]:
## 统计用户年购物频次,购物总金额,购物时间间隔均值,方差
def des_year(year):
    df_sub = pd.DataFrame()
    order_sub = data[data['ordTime'].dt.year==year]
    tmp1 = order_sub.groupby(by=['uid'])['ordTime'].describe()
    df_sub['order_count_'+str(year)] = tmp1['unique']
    df_sub['actualFee_sum_'+str(year)] = order_sub.groupby(by=['uid'])['actualFee'].sum()
    df_sub['quantity_sum_'+str(year)] = order_sub.groupby(by=['uid'])['quantity'].sum()
    tmp1 = order_sub.groupby(by=['uid'])['ordTime'].apply(time_gap)
    df_sub['order_time_gapMean_'+str(year)] = tmp1.apply(lambda x:x[0])
    df_sub['order_time_gapStd_'+str(year)] = tmp1.apply(lambda x:x[1])
    return df_sub

#2016-2017年统计结果
result_year = pd.DataFrame(index=result1.index)
for i in xrange(2016,2018,1):
    tmp =  des_year(i)
    result_year =result_year.merge(tmp,left_index=True,right_index=True,how='left')
    print result_year.shape

In [27]:
#merge
result1 = result1.merge(result_year,left_index=True,right_index=True,how='left')
result1.to_csv('./data/feature_usr_base_17w.csv')
result1.shape,result1.head()

((170054, 41),
                                   actualFee_sum  actualFee_mean  \
 uid                                                               
 00006f133dc9233b5476ed47b00aa838        2774.06       71.129744   
 0001101072642d28cbec77255a351fa1         317.60       52.933333   
 000120625f722f19873e3e08ef3ee0d7         330.80       55.133333   
 000166ec0b788bb54690c172e5946574        1383.90       44.641935   
 0001c5790371fcaec5b9aa8327a905be        5050.85       67.344667   
 
                                   actualFee_median  actualFee_std  \
 uid                                                                 
 00006f133dc9233b5476ed47b00aa838              39.9      76.600683   
 0001101072642d28cbec77255a351fa1              57.4      33.439597   
 000120625f722f19873e3e08ef3ee0d7              47.1      45.429095   
 000166ec0b788bb54690c172e5946574              34.0      35.564498   
 0001c5790371fcaec5b9aa8327a905be              45.0     100.508308   
 
               

In [68]:
# result1['huabeiTotal'].describe()
result1['actualFee_sum_2016'].describe()
result1['actualFee_sum_2017'].describe()

count    156045.000000
mean       1512.074432
std        3493.806911
min           0.000000
25%         227.000000
50%         662.000000
75%        1669.600000
max      506916.860000
Name: actualFee_sum_2017, dtype: float64

In [40]:
result1.corr()['huabeiTotal']

actualFee_sum              0.254433
actualFee_mean             0.024122
actualFee_median           0.002630
actualFee_std              0.084429
actualFee_min             -0.005930
actualFee_max              0.131734
quantity_sum               0.371527
quantity_mean              0.131368
quantity_median            0.066072
quantity_std               0.214854
quantity_min              -0.078351
quantity_max               0.282280
good_avg_fee               0.017816
order_count                0.326692
order_shop_most            0.120885
order_avg_fee              0.025157
shop_count                 0.321822
shop_freq_num              0.090782
shop_freq_ratio           -0.164243
online_days                0.107554
online_years               0.107708
year_avg_fee               0.276120
year_avg_order             0.292791
order_close_count          0.062115
order_close_ratio         -0.136453
free_post_count            0.313924
free_post_ratio           -0.014320
huabei_cost                0

### 2.店铺信息统计


In [55]:
#店铺信息统计,店铺购买次数,人数
tmp1 = data.groupby(by=['shopNick'])['uid'].describe()
shop = data[['shopNick']].drop_duplicates()
shop = shop.merge(tmp1[['unique','count']].reset_index(),on='shopNick',how='left')
del tmp1
shop

,shopNick,unique,count
0,nice1st,11,12
1,我爱悠享每刻,2,2
2,guohaotj,2,2
3,菜花岛主,9,76
4,杨记土味,4,6
5,佳琦母婴专营店,11,11
6,译海渊博旗舰店,28,28
7,卡兰薇旗舰店,248,259
8,菜叶儿,150,208
9,树博办公用品专营店,52,55


In [57]:
#店铺总销售额
shop_actualFee = data.groupby(by=['shopNick'])['actualFee'].sum()
shop = shop.merge(shop_actualFee.reset_index(),on='shopNick',how='left')
del shop_actualFee,shop_quantity
shop

,shopNick,unique,count,actualFee,quantity
0,nice1st,11,12,1881.66,1.916667
1,我爱悠享每刻,2,2,196.00,2.000000
2,guohaotj,2,2,17.60,1.000000
3,菜花岛主,9,76,2124.94,2.197368
4,杨记土味,4,6,507.40,2.500000
5,佳琦母婴专营店,11,11,703.00,1.909091
6,译海渊博旗舰店,28,28,845.65,1.428571
7,卡兰薇旗舰店,248,259,5269.92,1.285714
8,菜叶儿,150,208,15557.35,2.370192
9,树博办公用品专营店,52,55,1535.01,1.600000


In [58]:
shop = shop.rename(columns={'unique':'buy_person_count','count':'buy_count','actualFee':'actualFee_sum'})
shop

,shopNick,buy_person_count,buy_count,actualFee_sum,avg_order_quantity
0,nice1st,11,12,1881.66,1.916667
1,我爱悠享每刻,2,2,196.00,2.000000
2,guohaotj,2,2,17.60,1.000000
3,菜花岛主,9,76,2124.94,2.197368
4,杨记土味,4,6,507.40,2.500000
5,佳琦母婴专营店,11,11,703.00,1.909091
6,译海渊博旗舰店,28,28,845.65,1.428571
7,卡兰薇旗舰店,248,259,5269.92,1.285714
8,菜叶儿,150,208,15557.35,2.370192
9,树博办公用品专营店,52,55,1535.01,1.600000


In [59]:
shop['per_buy_count'] = 1.0*shop['buy_count']/shop['buy_person_count']
shop['per_avg_fee'] = 1.0*shop['actualFee_sum']/shop['buy_person_count']
shop['avg_order_fee'] = 1.0*shop['actualFee_sum']/shop['buy_count']
shop.shape,shop.head()

((983677, 8),
    shopNick buy_person_count buy_count  actualFee_sum  avg_order_quantity  \
 0   nice1st               11        12        1881.66            1.916667   
 1    我爱悠享每刻                2         2         196.00            2.000000   
 2  guohaotj                2         2          17.60            1.000000   
 3      菜花岛主                9        76        2124.94            2.197368   
 4      杨记土味                4         6         507.40            2.500000   
 
   per_buy_count per_avg_fee avg_order_fee  
 0       1.09091      171.06       156.805  
 1             1          98            98  
 2             1         8.8           8.8  
 3       8.44444     236.104       27.9597  
 4           1.5      126.85       84.5667  )

In [63]:
#save
shop.to_csv('./data/shop_feature_17w.csv',index=None)
shop

,shopNick,buy_person_count,buy_count,actualFee_sum,per_buy_count,per_avg_fee,avg_order_fee
758363,唐僧洗头用霸王369,1,1,246.90,1,246.9,246.9
970769,阚阚小屋1,1,1,133.92,1,133.92,133.92
588803,鲜橙鱼鱼,1,1,82.20,1,82.2,82.2
848950,caidanshuang1986,2,3,3174.08,1.5,1587.04,1058.03
273081,liuluoxing,1,1,235.11,1,235.11,235.11
957093,cgfwxj,1,1,55.21,1,55.21,55.21
449452,爱上饰品5,1,1,97.78,1,97.78,97.78
710848,薇2安娜,1,1,567.69,1,567.69,567.69
854660,一分利时代,1,1,180.37,1,180.37,180.37
687922,精品多肉种子铺,1,1,85.14,1,85.14,85.14


##  商品数据统计

In [3]:
goods = pd.read_csv('./data/goods_17w.csv')
goods = goods.drop_duplicates()
goods.shape,goods.head()

((11605101, 11),
                                 uid shopNick   shopName      shopId  \
 0  3824d0d90e953210dc6947175784a6ff    素柏旗舰店      素柏旗舰店  1739811044   
 1  3824d0d90e953210dc6947175784a6ff    素柏旗舰店      素柏旗舰店  1739811044   
 2  3824d0d90e953210dc6947175784a6ff   笑花落了谁家     格调原创女鞋   411155310   
 3  3824d0d90e953210dc6947175784a6ff   笑花落了谁家     格调原创女鞋   411155310   
 4  3824d0d90e953210dc6947175784a6ff      舍之秀  丫仔日记 潮品童装   853501992   
 
               ordNum  quantity           goodsSize  \
 0  20975380035224246         2      颜色分类:浅蓝色;尺寸:28   
 1  20975380035224246         2                 NaN   
 2  19235628080224246         2   颜色分类:白色;尺码:39[偏小]   
 3  19235628080224246         2                 NaN   
 4  18872368348224246         2  颜色分类:白色;参考身高:100cm   
 
                              goodsName promotionPrice originalPrice goodsNum  
 0     2017新款高腰修身牛仔短裤女夏宽松韩版流苏毛边大码学生阔腿热裤             49           399        1  
 1                                 保险服务            NaN   

In [9]:
#用户购买折扣商品数
goods['is_discount'] = goods.promotionPrice!=goods.originalPrice
goods['is_discount']=goods['is_discount'].map(lambda x: 1 if x else 0)
goods.loc[goods[goods.goodsName=='保险服务'].index,'is_discount']=0
tmp1 = goods.groupby(by=['uid'])['is_discount'].apply(sum)
tmp1.name = 'discount_count'
tmp1 = tmp1.reset_index()
tmp1.head()

,uid,discount_count
0,00006f133dc9233b5476ed47b00aa838,33
1,0001101072642d28cbec77255a351fa1,6
2,000120625f722f19873e3e08ef3ee0d7,5
3,000166ec0b788bb54690c172e5946574,19
4,0001c5790371fcaec5b9aa8327a905be,73


In [7]:
#付费运费险订单数
tmp2 = goods[(goods.goodsName=='保险服务') & (goods.originalPrice>0)]
tmp = tmp2.groupby(by=['uid'])['originalPrice'].count()
tmp.name ='baoxian_fee_count' 
tmp.head()

uid
00022dd34d5ab0b86d0b93676df27da9     6
0007652a90d92e7c3648b604ca107086    36
000aa389d918375d87320f871990620f     1
000aba1e2494f096ee47e7b88f91acc3    20
000f060ff2443a00ce99ef1d15b840a7     1
Name: baoxian_fee_count, dtype: int64

In [20]:
#保存结果
tmp1 = tmp1.merge(tmp.reset_index(),on='uid',how='left')
tmp1.fillna(0,inplace=True)
tmp1.to_csv('./data/feature_usr_other_17w.csv',index=None)
tmp1.head()

,uid,discount_count,baoxian_fee_count
0,00006f133dc9233b5476ed47b00aa838,33,0.0
1,0001101072642d28cbec77255a351fa1,6,0.0
2,000120625f722f19873e3e08ef3ee0d7,5,0.0
3,000166ec0b788bb54690c172e5946574,19,0.0
4,0001c5790371fcaec5b9aa8327a905be,73,0.0


## 合并所有用户特征

In [64]:
data1 = pd.read_csv('./data/feature_usr_base_17w.csv')
data2 = pd.read_csv('./data/feature_usr_other_17w.csv')
data1.shape,data2.shape

((170054, 42), (176870, 3))

In [65]:
data1 = data1.merge(data2,on='uid',how='left')
data1.to_csv('./data/feature_usr_base.csv',index=None)
data1.shape,data1.head()

((170054, 44),
                                 uid  actualFee_sum  actualFee_mean  \
 0  00006f133dc9233b5476ed47b00aa838        2774.06       71.129744   
 1  0001101072642d28cbec77255a351fa1         317.60       52.933333   
 2  000120625f722f19873e3e08ef3ee0d7         330.80       55.133333   
 3  000166ec0b788bb54690c172e5946574        1383.90       44.641935   
 4  0001c5790371fcaec5b9aa8327a905be        5050.85       67.344667   
 
    actualFee_median  actualFee_std  actualFee_min  actualFee_max  \
 0              39.9      76.600683           7.96          399.0   
 1              57.4      33.439597           9.90           89.0   
 2              47.1      45.429095           9.80          133.0   
 3              34.0      35.564498           9.90          158.0   
 4              45.0     100.508308           8.50          798.0   
 
    quantity_sum  quantity_mean  quantity_median        ...         \
 0          88.0       2.256410              2.0        ...          
 

## 模型训练数据

In [47]:
# 花呗额度预测数据
ab_buy = pd.read_csv('./data/usr_buy_abGoods.csv')
data1 = data1.merge(ab_buy,on='uid',how='left')
data1.fillna(0,inplace=True)
data1

,uid,actualFee_sum,actualFee_mean,actualFee_median,actualFee_std,actualFee_min,actualFee_max,quantity_sum,quantity_mean,quantity_median,...,year_avg_fee,year_avg_order,order_close_count,order_close_ratio,huabei_cost,huabei_cost_ratio,huabeiTotal,discount_count,baoxian_fee_count,buy_abnormal_count
0,00006f133dc9233b5476ed47b00aa838,2774.06,71.129744,39.900,76.600683,7.96,399.00,88.0,2.256410,2.0,...,1387.030000,19.500000,9,0.230769,0.00,0.000000,0.0,33,0.0,0.0
1,0001101072642d28cbec77255a351fa1,317.60,52.933333,57.400,33.439597,9.90,89.00,12.0,2.000000,2.0,...,317.600000,6.000000,1,0.166667,215.57,0.757875,500.0,6,0.0,0.0
2,000120625f722f19873e3e08ef3ee0d7,330.80,55.133333,47.100,45.429095,9.80,133.00,13.0,2.166667,2.0,...,330.800000,3.000000,0,0.000000,17.21,0.524695,50.0,5,0.0,0.0
3,000166ec0b788bb54690c172e5946574,1383.90,44.641935,34.000,35.564498,9.90,158.00,93.0,3.000000,2.0,...,461.300000,10.333333,1,0.032258,0.00,0.000000,0.0,19,0.0,0.0
4,0001c5790371fcaec5b9aa8327a905be,5050.85,67.344667,45.000,100.508308,8.50,798.00,114.0,1.520000,1.0,...,1683.616667,25.000000,14,0.186667,0.00,0.000000,4000.0,73,0.0,0.0
5,0002155afc87ff5734243956f84fae8a,39.40,19.700000,19.700,5.374012,15.90,23.50,2.0,1.000000,1.0,...,39.400000,2.000000,0,0.000000,69.30,97.605634,70.0,2,0.0,0.0
6,00022dd34d5ab0b86d0b93676df27da9,3870.47,56.093768,26.500,64.603769,9.80,361.42,136.0,1.971014,2.0,...,3870.470000,69.000000,24,0.347826,5003.81,52.014657,5100.0,47,6.0,0.0
7,00024bf12d506f5d9d8e2780c39e9976,2594.42,44.731379,24.900,66.894445,7.90,445.00,89.0,1.534483,1.0,...,1297.210000,29.000000,9,0.155172,1499.64,4053.081081,1500.0,46,0.0,0.0
8,00026bf5b1403838a7fe4cf50bd92b69,49.00,49.000000,49.000,0.000000,49.00,49.00,1.0,1.000000,1.0,...,49.000000,1.000000,0,0.000000,68.96,65.676190,70.0,1,0.0,0.0
9,00027f72100e9f2803dc48b3b5e4ac75,1771.27,38.505870,27.450,37.367426,5.80,226.27,98.0,2.130435,1.5,...,885.635000,22.500000,8,0.177778,119.00,0.030662,4000.0,66,0.0,0.0


In [62]:
cols =[col for col in list(data1.columns) if 'huabei' not in col]
cols.append('huabeiTotal')
samples = data1.ix[:,cols]
samples = samples[samples.huabeiTotal>0]
samples = samples.set_index(['uid'])
samples.to_csv('./model_data/huabei_pre.csv')
samples.head()

,actualFee_sum,actualFee_mean,actualFee_median,actualFee_std,actualFee_min,actualFee_max,quantity_sum,quantity_mean,quantity_median,quantity_std,...,online_days,online_years,year_avg_fee,year_avg_order,order_close_count,order_close_ratio,discount_count,baoxian_fee_count,buy_abnormal_count,huabeiTotal
uid,,,,,,,,,,,,,,,,,,,,,
0001101072642d28cbec77255a351fa1,317.60,52.933333,57.4,33.439597,9.9,89.00,12.0,2.000000,2.0,0.632456,...,63,1,317.600000,6.0,1,0.166667,6,0.0,0.0,500.0
000120625f722f19873e3e08ef3ee0d7,330.80,55.133333,47.1,45.429095,9.8,133.00,13.0,2.166667,2.0,0.408248,...,113,1,330.800000,3.0,0,0.000000,5,0.0,0.0,50.0
0001c5790371fcaec5b9aa8327a905be,5050.85,67.344667,45.0,100.508308,8.5,798.00,114.0,1.520000,1.0,0.577974,...,794,3,1683.616667,25.0,14,0.186667,73,0.0,0.0,4000.0
0002155afc87ff5734243956f84fae8a,39.40,19.700000,19.7,5.374012,15.9,23.50,2.0,1.000000,1.0,0.000000,...,28,1,39.400000,2.0,0,0.000000,2,0.0,0.0,70.0
00022dd34d5ab0b86d0b93676df27da9,3870.47,56.093768,26.5,64.603769,9.8,361.42,136.0,1.971014,2.0,1.236349,...,186,1,3870.470000,69.0,24,0.347826,47,6.0,0.0,5100.0


In [74]:
huabei = data1['huabeiTotal'].value_counts().sort_index()
huabei.name= 'per_count'
huabei = huabei.reset_index()
huabei.rename(columns={'index':'huabeiTotal'},inplace= True)
huabei[huabei.huabeiTotal<10000]


,huabeiTotal,per_count
0,0.0,30934
1,50.0,1355
2,70.0,7148
3,100.0,441
4,200.0,2329
5,300.0,686
6,400.0,2
7,500.0,19555
8,600.0,55
9,700.0,120
